In [4]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

def plot_mapa_calor_interativo(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário, filtrando por intervalo de datas,
    e tratando outliers.
    
    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap.html").
    """
    # Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # Verifique se a coluna 'geometry' está corretamente interpretada
    df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])

    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Agrupar os dados por setor censitário, somando as notificações
    gdf_agrupado = gdf.groupby('censitario').agg({
        'notificacoes': 'sum',
        'geometry': 'first'  # Preservar a geometria do setor censitário
    }).reset_index()

    # Criar um GeoDataFrame para visualização
    gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # Identificar outliers
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # Separar colunas para escala de cores
    gdf_agrupado['color'] = gdf_agrupado['is_outlier'].apply(lambda x: 'Outlier' if x else 'Normal')

    # Extrair coordenadas para o Plotly
    gdf_agrupado['lon'] = gdf_agrupado.geometry.centroid.x
    gdf_agrupado['lat'] = gdf_agrupado.geometry.centroid.y

    # Configurar o centro e o zoom do mapa com base nos setores censitários
    center_lat = gdf_agrupado['lat'].mean()
    center_lon = gdf_agrupado['lon'].mean()

    # Criar mapa interativo com Plotly
    fig = px.scatter_mapbox(
        gdf_agrupado,
        lat='lat',
        lon='lon',
        color='is_outlier',
        size='notificacoes',
        size_max=15,
        mapbox_style="carto-positron",
        color_discrete_map={True: "black", False: "orange"},
        hover_data={'notificacoes': True, 'censitario': True},
        title="Mapa de Calor - Notificações em São José do Rio Preto",
        center=dict(lat=center_lat, lon=center_lon),
        zoom=12
    )

    # Ajustar a opacidade e outras configurações
    fig.update_traces(marker=dict(opacity=0.7))

    # Salvar como HTML
    fig.write_html(output_html)
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso:
parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

# Intervalo de datas para o filtro
data_inicio = "202201"  # Data inicial no formato YYYYMM
data_fim = "202212"     # Data final no formato YYYYMM

# Gerar o mapa
plot_mapa_calor_interativo(parquet_file, data_inicio=data_inicio, data_fim=data_fim)


Mapa salvo como heatmap.html


In [14]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

def plot_mapa_calor_interativo(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário, filtrando por intervalo de datas,
    e tratando outliers.
    
    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap.html").
    """
    # Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # Verifique se a coluna 'geometry' está corretamente interpretada
    df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])

    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Agrupar os dados por setor censitário, somando as notificações
    gdf_agrupado = gdf.groupby('censitario').agg({
        'notificacoes': 'sum',
        'geometry': 'first'  # Preservar a geometria do setor censitário
    }).reset_index()

    # Criar um GeoDataFrame para visualização
    gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # Identificar outliers e criar uma coluna de flag
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # Extrair coordenadas para o Plotly
    gdf_agrupado['lon'] = gdf_agrupado.geometry.centroid.x
    gdf_agrupado['lat'] = gdf_agrupado.geometry.centroid.y

    # Configurar o centro e o zoom do mapa com base nos setores censitários
    center_lat = gdf_agrupado['lat'].mean()
    center_lon = gdf_agrupado['lon'].mean()

    # Separar outliers e não outliers
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # Criar mapa interativo com Plotly para dados normais
    fig = px.scatter_mapbox(
        gdf_normal,
        lat='lat',
        lon='lon',
        color='notificacoes',
        size='notificacoes',
        size_max=15,
        mapbox_style="carto-positron",
        color_continuous_scale="Reds",
        hover_data={'notificacoes': True, 'censitario': True},
        title="Mapa de Calor - Notificações de Dengue em São José do Rio Preto",
        center=dict(lat=center_lat, lon=center_lon),
        zoom=12
    )
    
    # Definir o nome da primeira trace
    if len(fig.data) > 0:
        fig.data[0].name = 'Notificações'

    # Adicionar outliers com cor preta
    if not gdf_outliers.empty:
        outlier_trace = px.scatter_mapbox(
            gdf_outliers,
            lat='lat',
            lon='lon',
            size='notificacoes',
            size_max=15,
            mapbox_style="carto-positron",
            hover_data={'notificacoes': True, 'censitario': True}
        ).data[0]
        # Atualizar a cor dos outliers para preto
        outlier_trace.marker.color = 'black'
        outlier_trace.name = 'Outliers'
        # Ajustar propriedades adicionais se necessário
        outlier_trace.marker.sizemode = 'area'
        outlier_trace.marker.opacity = 0.7
        fig.add_trace(outlier_trace)

    # Atualizar layout para lidar com legendas
    fig.update_layout(
        legend=dict(
            title="Legenda",
            itemsizing='constant'
        )
    )

    # Salvar como HTML
    fig.write_html(output_html)
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso:
parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

# Intervalo de datas para o filtro
data_inicio = "202201"  # Data inicial no formato YYYYMM
data_fim = "202212"     # Data final no formato YYYYMM

# Gerar o mapa
plot_mapa_calor_interativo(parquet_file, data_inicio=data_inicio, data_fim=data_fim)


Mapa salvo como heatmap.html


In [7]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

def plot_mapa_calor_interativo(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário dentro de São José do Rio Preto,
    filtrando por intervalo de datas e tratando outliers.
    
    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap.html").
    """
    # Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # Verifique se a coluna 'geometry' está corretamente interpretada
    df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])

    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Agrupar os dados por setor censitário, somando as notificações
    gdf_agrupado = gdf.groupby('censitario').agg({
        'notificacoes': 'sum',
        'geometry': 'first'  # Preservar a geometria do setor censitário
    }).reset_index()

    # Simplificar geometrias para melhorar a performance
    # Ajuste o parâmetro 'tolerance' conforme necessário
    gdf_agrupado['geometry'] = gdf_agrupado['geometry'].simplify(tolerance=0.001, preserve_topology=True)

    # Criar um GeoDataFrame para visualização
    gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # Identificar outliers e criar uma coluna de flag
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # Separar outliers e não outliers
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # Calcular o centro do município para centralizar o mapa
    # Usando unary_union para garantir que estamos pegando o centro do conjunto completo
    centro = gdf_agrupado.geometry.unary_union.centroid
    center_lat = centro.y
    center_lon = centro.x

    # Preparar GeoJSON para choropleth_mapbox
    # Adicionar um ID único correspondente à coluna 'censitario'
    gdf_normal = gdf_normal.copy()
    gdf_normal['censitario_str'] = gdf_normal['censitario'].astype(str)
    geojson_normal = gdf_normal.__geo_interface__

    # Criar mapa de coropleta para dados normais
    fig = px.choropleth_mapbox(
        gdf_normal,
        geojson=geojson_normal,
        locations='censitario_str',
        color='notificacoes',
        color_continuous_scale="Reds",
        mapbox_style="carto-positron",
        center={"lat": center_lat, "lon": center_lon},
        zoom=12,
        opacity=0.7,
        hover_data={'notificacoes': True, 'censitario': True},
        labels={'notificacoes': 'Notificações'},
        featureidkey="properties.censitario_str",
        title="Mapa de Calor - Notificações em São José do Rio Preto"
    )

    # Adicionar camada de outliers com cor preta
    if not gdf_outliers.empty:
        gdf_outliers = gdf_outliers.copy()
        gdf_outliers['censitario_str'] = gdf_outliers['censitario'].astype(str)
        geojson_outliers = gdf_outliers.__geo_interface__

        fig.add_trace(
            px.choropleth_mapbox(
                gdf_outliers,
                geojson=geojson_outliers,
                locations='censitario_str',
                color_discrete_sequence=['black'],
                mapbox_style="carto-positron",
                opacity=0.9,
                hover_data={'notificacoes': True, 'censitario': True},
                labels={'notificacoes': 'Notificações'},
                featureidkey="properties.censitario_str"
            ).data[0]
        )
        # Atualizar o nome da trace de outliers
        fig.data[-1].name = 'Outliers'

    # Atualizar layout para melhorar a legenda
    fig.update_layout(
        legend=dict(
            title="Legenda",
            itemsizing='constant'
        )
    )

    # Salvar como HTML
    fig.write_html(output_html)
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso:
parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

# Intervalo de datas para o filtro
data_inicio = "202201"  # Data inicial no formato YYYYMM
data_fim = "202212"     # Data final no formato YYYYMM

# Gerar o mapa
plot_mapa_calor_interativo(parquet_file, data_inicio=data_inicio, data_fim=data_fim)


AttributeError: 'Series' object has no attribute 'simplify'

In [8]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

def plot_mapa_calor_interativo(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap.html",
    crs_original="EPSG:4674"  # Defina o CRS original dos seus dados
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário dentro de São José do Rio Preto,
    filtrando por intervalo de datas e tratando outliers.

    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap.html").
        crs_original (str): Sistema de Referência de Coordenadas original dos dados (default: "EPSG:4674").
    """
    # 1. Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # 2. Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # 3. Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # 4. Verificar e converter a coluna 'geometry' de WKB para GeoSeries
    df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])

    # 5. Converter para GeoDataFrame e definir o CRS original
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=crs_original)

    # 6. Agrupar os dados por setor censitário, somando as notificações
    gdf_agrupado = gdf.groupby('censitario').agg({
        'notificacoes': 'sum',
        'geometry': 'first'  # Preservar a geometria do setor censitário
    }).reset_index()

    # 7. Converter para GeoDataFrame imediatamente após o groupby
    gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry', crs=gdf.crs)

    # 8. Converter para CRS WGS84 se necessário (Plotly trabalha melhor com EPSG:4326)
    if gdf_agrupado.crs != "EPSG:4326":
        gdf_agrupado = gdf_agrupado.to_crs("EPSG:4326")

    # 9. Simplificar geometrias para melhorar a performance
    # Ajuste o parâmetro 'tolerance' conforme necessário
    gdf_agrupado['geometry'] = gdf_agrupado['geometry'].simplify(tolerance=0.001, preserve_topology=True)

    # 10. Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # 11. Identificar outliers e criar uma coluna de flag
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # 12. Separar outliers e não outliers
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # 13. Calcular o centro do município para centralizar o mapa
    # Usando union_all() para evitar a depreciação do unary_union
    centro = gdf_agrupado.geometry.union_all().centroid
    center_lat = centro.y
    center_lon = centro.x

    # 14. Preparar GeoJSON para choropleth_mapbox
    # Adicionar um ID único correspondente à coluna 'censitario_str'
    gdf_normal = gdf_normal.copy()
    gdf_normal['censitario_str'] = gdf_normal['censitario'].astype(str)
    gdf_normal = gdf_normal[['censitario_str', 'notificacoes', 'geometry']]
    geojson_normal = gdf_normal.__geo_interface__

    # 15. Criar mapa de coropleta para dados normais
    fig = px.choropleth_mapbox(
        gdf_normal,
        geojson=geojson_normal,
        locations='censitario_str',
        color='notificacoes',
        color_continuous_scale="Reds",
        mapbox_style="carto-positron",
        center={"lat": center_lat, "lon": center_lon},
        zoom=12,
        opacity=0.7,
        hover_data={'notificacoes': True, 'censitario_str': True},
        labels={'notificacoes': 'Notificações', 'censitario_str': 'Setor Censitário'},
        featureidkey="properties.censitario_str",
        title="Mapa de Calor - Notificações de Dengue em São José do Rio Preto"
    )

    # 16. Adicionar camada de outliers com cor preta
    if not gdf_outliers.empty:
        gdf_outliers = gdf_outliers.copy()
        gdf_outliers['censitario_str'] = gdf_outliers['censitario'].astype(str)
        gdf_outliers = gdf_outliers[['censitario_str', 'notificacoes', 'geometry']]
        geojson_outliers = gdf_outliers.__geo_interface__

        # Criar choropleth para outliers com cor preta
        fig.add_trace(
            px.choropleth_mapbox(
                gdf_outliers,
                geojson=geojson_outliers,
                locations='censitario_str',
                color_discrete_sequence=['black'],
                mapbox_style="carto-positron",
                opacity=0.9,
                hover_data={'notificacoes': True, 'censitario_str': True},
                labels={'notificacoes': 'Notificações', 'censitario_str': 'Setor Censitário'},
                featureidkey="properties.censitario_str"
            ).data[0]
        )
        # Atualizar o nome da trace de outliers
        fig.data[-1].name = 'Outliers'

    # 17. Atualizar layout para melhorar a legenda
    fig.update_layout(
        legend=dict(
            title="Legenda",
            itemsizing='constant'
        )
    )

    # 18. Salvar como HTML
    fig.write_html(output_html)
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso:
parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

# Intervalo de datas para o filtro
data_inicio = "202201"  # Data inicial no formato YYYYMM
data_fim = "202212"     # Data final no formato YYYYMM

# Gerar o mapa
plot_mapa_calor_interativo(
    parquet_file,
    data_inicio=data_inicio,
    data_fim=data_fim,
    crs_original="EPSG:4674"  # Ajuste conforme o CRS dos seus dados
)


Mapa salvo como heatmap.html


In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

def plot_mapa_calor_interativo(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap_plotly_correct.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário, filtrando por intervalo de datas,
    e tratando outliers.
    
    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap.html").
    """
    # Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # Verifique se a coluna 'geometry' está corretamente interpretada
    df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])

    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Agrupar os dados por setor censitário, somando as notificações
    gdf_agrupado = gdf.groupby('censitario').agg({
        'notificacoes': 'sum',
        'geometry': 'first'  # Preservar a geometria do setor censitário
    }).reset_index()

    # Criar um GeoDataFrame para visualização
    gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # Identificar outliers e criar uma coluna de flag
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # Extrair coordenadas para o Plotly
    gdf_agrupado['lon'] = gdf_agrupado.geometry.centroid.x
    gdf_agrupado['lat'] = gdf_agrupado.geometry.centroid.y

    # Configurar o centro e o zoom do mapa com base nos setores censitários
    center_lat = gdf_agrupado['lat'].mean()
    center_lon = gdf_agrupado['lon'].mean()

    # Separar outliers e não outliers
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # Criar mapa interativo com Plotly para dados normais
    fig = px.scatter_mapbox(
        gdf_normal,
        lat='lat',
        lon='lon',
        color='notificacoes',
        size='notificacoes',
        size_max=15,
        mapbox_style="carto-positron",
        color_continuous_scale="Reds",
        hover_data={'notificacoes': True, 'censitario': True},
        title="Mapa de Calor - Notificações de Dengue em São José do Rio Preto",
        center=dict(lat=center_lat, lon=center_lon),
        zoom=12
    )
    
    # Definir o nome da primeira trace
    if len(fig.data) > 0:
        fig.data[0].name = 'Notificações'

    # Adicionar outliers com cor preta
    if not gdf_outliers.empty:
        outlier_trace = px.scatter_mapbox(
            gdf_outliers,
            lat='lat',
            lon='lon',
            size='notificacoes',
            size_max=15,
            mapbox_style="carto-positron",
            hover_data={'notificacoes': True, 'censitario': True}
        ).data[0]
        # Atualizar a cor dos outliers para preto
        outlier_trace.marker.color = 'black'
        outlier_trace.name = 'Outliers'
        # Ajustar propriedades adicionais se necessário
        outlier_trace.marker.sizemode = 'area'
        outlier_trace.marker.opacity = 0.7
        fig.add_trace(outlier_trace)

    # Atualizar layout para lidar com legendas
    fig.update_layout(
        legend=dict(
            title="Legenda",
            itemsizing='constant'
        )
    )

    # Salvar como HTML
    fig.write_html(output_html)
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso:
parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

# Intervalo de datas para o filtro
data_inicio = "202201"  # Data inicial no formato YYYYMM
data_fim = "202212"     # Data final no formato YYYYMM

# Gerar o mapa
plot_mapa_calor_interativo(parquet_file, data_inicio=data_inicio, data_fim=data_fim)


Mapa salvo como heatmap.html


In [29]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import linear
from shapely import wkb
import math

def plot_mapa_calor_interativo_folium(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap_folium.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário usando Folium,
    filtrando por intervalo de datas e tratando outliers.

    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap_folium.html").
    """
    # Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # Verifique se a coluna 'geometry' está corretamente interpretada
    # Supondo que a coluna 'geometry' esteja em formato WKB
    df['geometry'] = df['geometry'].apply(wkb.loads)

    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Garantir que o CRS está definido (ajuste conforme necessário)
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)  # Exemplo: WGS84
    else:
        gdf = gdf.to_crs(epsg=4326)  # Reprojetar para WGS84 se necessário

    # Agrupar os dados por setor censitário, somando as notificações e preservando a geometria
    # Usando 'dissolve' para garantir a preservação correta das geometrias
    try:
        gdf_agrupado = gdf.dissolve(by='censitario', aggfunc={'notificacoes': 'sum'})
    except TypeError as e:
        print(f"Erro ao executar dissolve: {e}")
        return

    # Resetar o índice para ter 'censitario' como coluna
    gdf_agrupado = gdf_agrupado.reset_index()

    # Garantir que 'gdf_agrupado' é um GeoDataFrame com a geometria correta
    if not isinstance(gdf_agrupado, gpd.GeoDataFrame):
        gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # Verificar e corrigir geometrias inválidas
    gdf_agrupado['geometry'] = gdf_agrupado['geometry'].buffer(0)

    # Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # Identificar outliers e criar uma coluna de flag
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # Calcular centroides
    gdf_agrupado['centroid'] = gdf_agrupado.geometry.centroid

    # Extrair coordenadas
    gdf_agrupado['lon'] = gdf_agrupado.centroid.x
    gdf_agrupado['lat'] = gdf_agrupado.centroid.y

    # (Opcional) Simplificar geometrias para melhorar a performance
    # Ajuste o parâmetro 'tolerance' conforme necessário
    # Garantir que a geometria é válida antes de simplificar
    gdf_agrupado['geometry'] = gdf_agrupado['geometry'].apply(lambda geom: geom.simplify(tolerance=0.001, preserve_topology=True) if geom.is_valid else geom)

    # Remover linhas com NaN em 'lat' ou 'lon'
    initial_count = len(gdf_agrupado)
    gdf_agrupado = gdf_agrupado.dropna(subset=['lat', 'lon'])
    final_count = len(gdf_agrupado)
    dropped_count = initial_count - final_count
    if dropped_count > 0:
        print(f"Removidas {dropped_count} linhas com coordenadas inválidas (NaN).")

    # Configurar o centro do mapa com base nos centroides
    center_lat = gdf_agrupado['lat'].mean()
    center_lon = gdf_agrupado['lon'].mean()

    # Separar outliers e não outliers
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # Criar o mapa base
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles='cartodbpositron')

    # Definir a escala de cores
    min_notif = gdf_normal['notificacoes'].min()
    max_notif = gdf_normal['notificacoes'].max()
    colormap = linear.Reds_09.scale(min_notif, max_notif)
    colormap.caption = 'Número de Notificações'
    colormap.add_to(m)

    # Adicionar marcadores para notificações normais
    for idx, row in gdf_normal.iterrows():
        # Verificar se 'lat' e 'lon' não são NaN
        if math.isnan(row['lat']) or math.isnan(row['lon']):
            continue  # Pular esta iteração se houver NaNs

        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,  # Ajuste o tamanho conforme necessário
            color=colormap(row['notificacoes']),
            fill=True,
            fill_color=colormap(row['notificacoes']),
            fill_opacity=0.7,
            popup=folium.Popup(f"Censitário: {row['censitario']}<br>Notificações: {row['notificacoes']}", max_width=250)
        ).add_to(m)

    # Adicionar marcadores para outliers
    for idx, row in gdf_outliers.iterrows():
        # Verificar se 'lat' e 'lon' não são NaN
        if math.isnan(row['lat']) or math.isnan(row['lon']):
            continue  # Pular esta iteração se houver NaNs

        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=7,  # Um pouco maior para destacar
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.9,
            popup=folium.Popup(f"Censitário: {row['censitario']}<br>Notificações (Outlier): {row['notificacoes']}", max_width=250)
        ).add_to(m)

    # Adicionar camada de controle para ligar/desligar outliers
    folium.LayerControl().add_to(m)

    # Salvar como HTML
    m.save(output_html)
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso:
if __name__ == "__main__":
    parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

    # Intervalo de datas para o filtro
    data_inicio = "202201"  # Data inicial no formato YYYYMM
    data_fim = "202212"     # Data final no formato YYYYMM

    # Gerar o mapa
    plot_mapa_calor_interativo_folium(
        parquet_file, 
        outlier_threshold_factor=3, 
        data_inicio=data_inicio, 
        data_fim=data_fim,
        output_html="heatmap_folium.html"
    )


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\3801905559.py:79: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\3801905559.py:82: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\3801905559.py:83: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




Removidas 2 linhas com coordenadas inválidas (NaN).
Mapa salvo como heatmap_folium.html


In [33]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import linear
from shapely import wkb
import math

def plot_mapa_calor_interativo_folium(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap_folium.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário usando Folium,
    filtrando por intervalo de datas e tratando outliers.

    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap_folium.html").
    """
    # Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # Verifique se a coluna 'geometry' está corretamente interpretada
    # Supondo que a coluna 'geometry' esteja em formato WKB
    df['geometry'] = df['geometry'].apply(wkb.loads)

    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Garantir que o CRS está definido (ajuste conforme necessário)
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)  # Exemplo: WGS84
    else:
        gdf = gdf.to_crs(epsg=4326)  # Reprojetar para WGS84 se necessário

    # Agrupar os dados por setor censitário, somando as notificações e preservando a geometria
    try:
        gdf_agrupado = gdf.dissolve(by='censitario', aggfunc={'notificacoes': 'sum'})
    except TypeError as e:
        print(f"Erro ao executar dissolve: {e}")
        return

    # Resetar o índice para ter 'censitario' como coluna
    gdf_agrupado = gdf_agrupado.reset_index()

    # Garantir que 'gdf_agrupado' é um GeoDataFrame com a geometria correta
    if not isinstance(gdf_agrupado, gpd.GeoDataFrame):
        gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # Verificar e corrigir geometrias inválidas
    gdf_agrupado['geometry'] = gdf_agrupado['geometry'].buffer(0)

    # Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # Identificar outliers e criar uma coluna de flag
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # Calcular centroides
    gdf_agrupado['centroid'] = gdf_agrupado.geometry.centroid

    # Extrair coordenadas
    gdf_agrupado['lon'] = gdf_agrupado.centroid.x
    gdf_agrupado['lat'] = gdf_agrupado.centroid.y

    # (Opcional) Simplificar geometrias para melhorar a performance
    # Ajuste o parâmetro 'tolerance' conforme necessário
    # Garantir que a geometria é válida antes de simplificar
    gdf_agrupado['geometry'] = gdf_agrupado['geometry'].apply(
        lambda geom: geom.simplify(tolerance=0.001, preserve_topology=True) if geom.is_valid else geom
    )

    # Remover linhas com NaN em 'lat' ou 'lon'
    initial_count = len(gdf_agrupado)
    gdf_agrupado = gdf_agrupado.dropna(subset=['lat', 'lon'])
    final_count = len(gdf_agrupado)
    dropped_count = initial_count - final_count
    if dropped_count > 0:
        print(f"Removidas {dropped_count} linhas com coordenadas inválidas (NaN).")

    # Separar outliers e não outliers antes de calcular min_notif e max_notif
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # Calcular os limites geográficos (bounding box)
    if not gdf_agrupado.empty:
        min_lat = gdf_agrupado['lat'].min()
        max_lat = gdf_agrupado['lat'].max()
        min_lon = gdf_agrupado['lon'].min()
        max_lon = gdf_agrupado['lon'].max()
        map_bounds = [[min_lat, min_lon], [max_lat, max_lon]]
    else:
        # Definir padrões caso gdf_agrupado esteja vazio
        min_lat = min_lon = -90
        max_lat = max_lon = 90
        map_bounds = [[min_lat, min_lon], [max_lat, max_lon]]
        print("gdf_agrupado está vazio. Usando limites padrão para o mapa.")

    # Definir a escala de cores após separar gdf_normal
    if not gdf_normal.empty:
        min_notif = gdf_normal['notificacoes'].min()
        max_notif = gdf_normal['notificacoes'].max()
    else:
        # Definir valores padrão caso gdf_normal esteja vazio
        min_notif = 0
        max_notif = 1
        print("gdf_normal está vazio. Usando valores padrão para a escala de cores.")

    # Criar o mapa base centrado na média das coordenadas
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2
    m = folium.Map(location=[center_lat, center_lon],
                   zoom_start=12,
                   tiles='cartodbpositron')

    # Definir a escala de cores
    colormap = linear.Reds_09.scale(min_notif, max_notif)
    colormap.caption = 'Número de Notificações'
    colormap.add_to(m)

    # Adicionar marcadores para notificações normais
    for idx, row in gdf_normal.iterrows():
        # Verificar se 'lat' e 'lon' não são NaN
        if math.isnan(row['lat']) or math.isnan(row['lon']):
            continue  # Pular esta iteração se houver NaNs

        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,  # Ajuste o tamanho conforme necessário
            color=colormap(row['notificacoes']),
            fill=True,
            fill_color=colormap(row['notificacoes']),
            fill_opacity=0.7,
            popup=folium.Popup(f"Censitário: {row['censitario']}<br>Notificações: {row['notificacoes']}", max_width=250)
        ).add_to(m)

    # Adicionar marcadores para outliers
    for idx, row in gdf_outliers.iterrows():
        # Verificar se 'lat' e 'lon' não são NaN
        if math.isnan(row['lat']) or math.isnan(row['lon']):
            continue  # Pular esta iteração se houver NaNs

        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=7,  # Um pouco maior para destacar
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.9,
            popup=folium.Popup(f"Censitário: {row['censitario']}<br>Notificações (Outlier): {row['notificacoes']}", max_width=250)
        ).add_to(m)

    # Ajustar o mapa para os limites dos dados
    if not gdf_agrupado.empty:
        m.fit_bounds(map_bounds)

    # Adicionar camada de controle para ligar/desligar outliers
    folium.LayerControl().add_to(m)

    # Salvar como HTML
    m.save(output_html)
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso:
if __name__ == "__main__":
    parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

    # Intervalo de datas para o filtro
    data_inicio = "202201"  # Data inicial no formato YYYYMM
    data_fim = "202212"     # Data final no formato YYYYMM

    # Gerar o mapa
    plot_mapa_calor_interativo_folium(
        parquet_file, 
        outlier_threshold_factor=3, 
        data_inicio=data_inicio, 
        data_fim=data_fim,
        output_html="heatmap_folium_3.html"
    )


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\1150507266.py:78: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\1150507266.py:81: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\1150507266.py:82: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




Removidas 2 linhas com coordenadas inválidas (NaN).
Mapa salvo como heatmap_folium_3.html


In [64]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from branca.colormap import linear
from shapely import wkb
import math
from folium.features import CustomIcon
from folium import IFrame

def plot_mapa_calor_interativo_folium(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap_folium.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário usando Folium,
    filtrando por intervalo de datas e tratando outliers, com tooltips personalizados.

    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap_folium.html").
    """
    # 1. Leitura do arquivo Parquet
    try:
        df = pd.read_parquet(parquet_file)
        print("Arquivo Parquet lido com sucesso.")
    except Exception as e:
        print(f"Erro ao ler o arquivo Parquet: {e}")
        return

    # 2. Garantir que a coluna 'mes_ano' é string para comparação
    if 'mes_ano' not in df.columns:
        print("A coluna 'mes_ano' não existe no DataFrame.")
        return

    df['mes_ano'] = df['mes_ano'].astype(str)

    # 3. Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
        print(f"Filtrado por data início: {data_inicio}")
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]
        print(f"Filtrado por data fim: {data_fim}")

    # 4. Verificar se a coluna 'geometry' está presente
    if 'geometry' not in df.columns:
        print("A coluna 'geometry' não existe no DataFrame.")
        return

    # 5. Converter geometria de WKB para objetos geométricos
    try:
        df['geometry'] = df['geometry'].apply(wkb.loads)
        print("Conversão da coluna 'geometry' concluída.")
    except Exception as e:
        print(f"Erro ao converter geometria: {e}")
        return

    # 6. Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # 7. Definir ou reprojetar o CRS para WGS84 (EPSG:4326)
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)
        print("CRS definido como EPSG:4326 (WGS84).")
    else:
        gdf = gdf.to_crs(epsg=4326)
        print("CRS reprojetado para EPSG:4326 (WGS84).")

    # 8. Agrupar os dados por setor censitário, somando as notificações
    try:
        gdf_agrupado = gdf.dissolve(by='censitario', aggfunc={'notificacoes': 'sum'})
        print("Agrupamento por 'censitario' concluído.")
    except TypeError as e:
        print(f"Erro ao executar dissolve: {e}")
        return

    # 9. Resetar o índice para ter 'censitario' como coluna
    gdf_agrupado = gdf_agrupado.reset_index()

    # 10. Garantir que 'gdf_agrupado' é um GeoDataFrame com a geometria correta
    if not isinstance(gdf_agrupado, gpd.GeoDataFrame):
        gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # 11. Corrigir geometrias inválidas
    gdf_agrupado['geometry'] = gdf_agrupado['geometry'].buffer(0)
    print("Geometrias corrigidas.")

    # 12. Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr
    print(f"Limiar de outlier calculado: {outlier_threshold}")

    # 13. Identificar outliers e criar uma coluna de flag
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold
    print("Outliers identificados.")

    # 14. Calcular centroides
    gdf_agrupado['centroid'] = gdf_agrupado.geometry.centroid

    # 15. Extrair coordenadas dos centroides
    gdf_agrupado['lon'] = gdf_agrupado.centroid.x
    gdf_agrupado['lat'] = gdf_agrupado.centroid.y

    # 16. Simplificar geometrias para melhorar a performance
    gdf_agrupado['geometry'] = gdf_agrupado['geometry'].apply(
        lambda geom: geom.simplify(tolerance=0.001, preserve_topology=True) if geom.is_valid else geom
    )
    print("Geometrias simplificadas.")

    # 17. Remover linhas com NaN em 'lat' ou 'lon'
    initial_count = len(gdf_agrupado)
    gdf_agrupado = gdf_agrupado.dropna(subset=['lat', 'lon'])
    final_count = len(gdf_agrupado)
    dropped_count = initial_count - final_count
    if dropped_count > 0:
        print(f"Removidas {dropped_count} linhas com coordenadas inválidas (NaN).")
    else:
        print("Nenhuma linha com coordenadas inválidas encontrada.")

    # 18. Separar outliers e não outliers
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # 19. Calcular os limites geográficos (bounding box) usando total_bounds
    if not gdf_agrupado.empty:
        min_lon, min_lat, max_lon, max_lat = gdf_agrupado.total_bounds
        map_bounds = [[min_lat, min_lon], [max_lat, max_lon]]
        print(f"Bounding Box calculado: {map_bounds}")
    else:
        # Definir padrões caso gdf_agrupado esteja vazio
        min_lat = min_lon = -90
        max_lat = max_lon = 90
        map_bounds = [[min_lat, min_lon], [max_lat, max_lon]]
        print("gdf_agrupado está vazio. Usando limites padrão para o mapa.")

    # 20. Definir a escala de cores após separar gdf_normal
    if not gdf_normal.empty:
        min_notif = gdf_normal['notificacoes'].min()
        max_notif = gdf_normal['notificacoes'].max()
        print(f"Notificações mínimas: {min_notif}, máximas: {max_notif}")
    else:
        # Definir valores padrão caso gdf_normal esteja vazio
        min_notif = 0
        max_notif = 1
        print("gdf_normal está vazio. Usando valores padrão para a escala de cores.")

    # 21. Criar o mapa base centrado na média das coordenadas
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2
    m = folium.Map(location=[center_lat, center_lon],
                   zoom_start=12,
                   tiles='cartodbpositron')
    print("Mapa base criado.")

    # 22. Definir a escala de cores
    colormap = linear.Reds_09.scale(min_notif, max_notif)
    colormap.caption = 'Número de Notificações'
    colormap.add_to(m)
    print("Escala de cores adicionada ao mapa.")

    # 23. Adicionar MarkerCluster para melhor performance
    marker_cluster = MarkerCluster(name='Notificações').add_to(m)
    outlier_cluster = MarkerCluster(name='Outliers').add_to(m)
    print("Clusters de marcadores adicionados.")

    # 24. Adicionar marcadores para notificações normais com Tooltip
    for idx, row in gdf_normal.iterrows():
        tooltip_text = f"Censitário: {row['censitario']}<br>Notificações: {row['notificacoes']}"
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,  # Ajuste o tamanho conforme necessário
            color=colormap(row['notificacoes']),
            fill=True,
            fill_color=colormap(row['notificacoes']),
            fill_opacity=0.7,
            popup=folium.Popup(tooltip_text, max_width=250),
            tooltip=tooltip_text,  # Adiciona o tooltip para hover
            **{'notificacoes': row['notificacoes']}  # Passa a quantidade de notificações para o JavaScript
        ).add_to(marker_cluster)

    print("Marcadores para notificações normais adicionados com Tooltips.")

    # 25. Adicionar marcadores para outliers com Tooltip
    for idx, row in gdf_outliers.iterrows():
        tooltip_text = f"Censitário: {row['censitario']}<br>Notificações (Outlier): {row['notificacoes']}"
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=7,  # Um pouco maior para destacar
            color='black',
            fill=True,
            fill_color='black',
            fill_opacity=0.9,
            popup=folium.Popup(tooltip_text, max_width=250),
            tooltip=tooltip_text,  # Adiciona o tooltip para hover
            **{'notificacoes': row['notificacoes']}  # Passa a quantidade de notificações para o JavaScript
        ).add_to(outlier_cluster)

    print("Marcadores para outliers adicionados com Tooltips.")

    # 26. Ajustar o mapa para os limites dos dados
    if not gdf_agrupado.empty:
        m.fit_bounds(map_bounds)
        print("Mapa ajustado para os limites dos dados.")
    else:
        print("Mapa mantido com os limites padrão.")

    # 27. Adicionar controle de camadas
    folium.LayerControl().add_to(m)
    print("Controle de camadas adicionado.")

    # 28. Adicionar Tooltips para Clusters
    # Folium não suporta diretamente tooltips para clusters, mas podemos adicionar um recurso
    # para exibir a soma das notificações no cluster ao passar o mouse.

    # Para isso, vamos adicionar um script JavaScript personalizado
    # que sobrepõe a função de criação de ícones de cluster para incluir a soma das notificações.

    # JavaScript para adicionar Tooltips aos Clusters
    cluster_tooltip_js = """
    <script>
    // Função para adicionar tooltips aos clusters
    function addClusterTooltips() {
        var markerClusters = null;
        for (var i in map._layers) {
            if (map._layers[i] instanceof L.MarkerClusterGroup) {
                markerClusters = map._layers[i];
                break;
            }
        }

        if (markerClusters !== null) {
            markerClusters.eachLayer(function(cluster) {
                // Calcula a soma das notificações dos marcadores dentro do cluster
                var markers = cluster.getAllChildMarkers();
                var sum = 0;
                markers.forEach(function(marker) {
                    sum += marker.options.notificacoes;
                });

                // Cria um tooltip com a soma
                var tooltipContent = "Soma de Notificações: " + sum;

                // Remove tooltip existente para evitar duplicatas
                cluster.unbindTooltip();

                // Adiciona o novo tooltip
                cluster.bindTooltip(tooltipContent, {permanent: false, direction: "top"});
            });
        }
    }

    // Adiciona tooltips ao carregar o mapa
    addClusterTooltips();

    // Atualiza tooltips quando o mapa é movido ou zoomed
    map.on('zoomend moveend', function(e) {
        addClusterTooltips();
    });

    // Atualiza tooltips quando um novo cluster é criado
    map.on('clusterclick', function(e) {
        addClusterTooltips();
    });
    </script>
    """

    # Adiciona o JavaScript ao mapa
    folium.Element(cluster_tooltip_js).add_to(m)
    print("JavaScript personalizado para Tooltips de Clusters adicionados.")

    # 29. Salvar como HTML
    try:
        m.save(output_html)
        print(f"Mapa salvo como {output_html}")
    except Exception as e:
        print(f"Erro ao salvar o mapa: {e}")

# Exemplo de uso:
if __name__ == "__main__":
    parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

    # Intervalo de datas para o filtro
    data_inicio = "202201"  # Data inicial no formato YYYYMM
    data_fim = "202212"     # Data final no formato YYYYMM

    # Gerar o mapa
    plot_mapa_calor_interativo_folium(
        parquet_file, 
        outlier_threshold_factor=3, 
        data_inicio=data_inicio, 
        data_fim=data_fim,
        output_html="heatmap_folium_2.html"
    )


Arquivo Parquet lido com sucesso.
Filtrado por data início: 202201
Filtrado por data fim: 202212
Conversão da coluna 'geometry' concluída.
CRS definido como EPSG:4326 (WGS84).
Agrupamento por 'censitario' concluído.
Geometrias corrigidas.
Limiar de outlier calculado: 70.0
Outliers identificados.
Geometrias simplificadas.
Removidas 2 linhas com coordenadas inválidas (NaN).
Bounding Box calculado: [[np.float64(-20.932274999), np.float64(-49.463267)], [np.float64(-20.6353369989999), np.float64(-49.255825)]]
Notificações mínimas: 1, máximas: 68
Mapa base criado.
Escala de cores adicionada ao mapa.
Clusters de marcadores adicionados.


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\55961187.py:107: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\55961187.py:110: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\celso\AppData\Local\Temp\ipykernel_11780\55961187.py:111: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




Marcadores para notificações normais adicionados com Tooltips.
Marcadores para outliers adicionados com Tooltips.
Mapa ajustado para os limites dos dados.
Controle de camadas adicionado.
JavaScript personalizado para Tooltips de Clusters adicionados.
Mapa salvo como heatmap_folium_2.html
